In [7]:

from math import inf as infinity
from random import choice
import platform
import time
import json


parent_child = {}
visited_list = []
terminals = []
Scores = {}
levels = {}
child_parent = {}
child_parent['---------'] = [None]

In [8]:
def stateGenerator(intial_state, pos, player):
    state = ""
    for i, row in enumerate(intial_state):
        if row != 'X' and row != 'O' and i == pos:
            state += player
        else:
            state += row
    if state == intial_state:
        return None
    return state
def Terminal_State(State):
    if State[0] == State[1] == State[2] =='O':
        return(True,-1)
    elif State[3] == State[4] == State[5] == 'O':
        return(True,-1)
    elif State[6] == State[7] == State[8] =='O':
        return(True,-1)
    elif State[0] == State[3] == State[6] =='O':
        return(True,-1)
    elif State[1] == State[4] == State[7] =='O':
        return(True,-1)
    elif State[2] == State[5] == State[8] =='O':
        return(True,-1)
    elif State[0] == State[4] == State[8] =='O':
        return(True,-1)
    elif State[2] == State[4] == State[6] =='O':
        return(True,-1)
    if State[0] == State[1] == State[2] =='X':
        return(True,1)
    elif State[3] == State[4] == State[5] == 'X':
        return(True,1)
    elif State[6] == State[7] == State[8] =='X':
        return(True,1)
    elif State[0] == State[3] == State[6] =='X':
        return(True,1)
    elif State[1] == State[4] == State[7] =='X':
        return(True,1)
    elif State[2] == State[5] == State[8] =='X':
        return(True,1)
    elif State[0] == State[4] == State[8] =='X':
        return(True,1)
    elif State[2] == State[4] == State[6] =='X':
        return(True,1)

    for i in State:
        if i =='-':
            return(False,0)
    return(True,0)
def isTerminal(state):
    return Terminal_State(state)[0]
def winning_score(state):
    return Terminal_State(state)[1]

In [4]:
def Tree(node):
    fringe = []
    fringe.append(node)
    while len(fringe) != 0:
        p = fringe.pop(0)
        yield p
        if (p.count('X') + p.count('O')) % 2 == 0:
            player = 'X'
            Scores[p] = [winning_score(p), 'O']
        else:
            player = 'O'
            Scores[p] = [winning_score(p), 'X']
        if not isTerminal(p):
            childs = generate_childs(p, player)
            parent_child[p] = childs
            for c in childs:
                fringe.append(c)
        else:
            if p not in terminals:
                terminals.append(p)
            parent_child[p] = childs

def generate_childs(node, player):
    children = []
    if player == 'X':
        for j, row in enumerate(node):
            state = stateGenerator(node, j, player)
            if state is not None:
                children.append(state)
                if state not in child_parent.keys():
                    child_parent[state] = []
                child_parent[state].append(node)
    else:
        for j, row in enumerate(node):
            state = stateGenerator(node, j, player)
            if state is not None:
                children.append(state)
                if state not in child_parent.keys():
                    child_parent[state] = []
                child_parent[state].append(node)

    return children


In [5]:
Scores['---------']=[0, 'O']
start_node = '---------'
k = list(Tree(start_node))
import json
with open('Tree.json', 'w+') as f:
    json.dump(parent_child, f)

with open('child_parent.json', 'w+') as f:
    json.dump(child_parent, f)


In [9]:


HUMAN = -1
COMP = +1
board = [
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
]


def evaluate(state):
    if wins(state, COMP):
        score = +1
    elif wins(state, HUMAN):
        score = -1
    else:
        score = 0

    return score

def wins(state, player):
    win_state = [
        [state[0][0], state[0][1], state[0][2]],
        [state[1][0], state[1][1], state[1][2]],
        [state[2][0], state[2][1], state[2][2]],
        [state[0][0], state[1][0], state[2][0]],
        [state[0][1], state[1][1], state[2][1]],
        [state[0][2], state[1][2], state[2][2]],
        [state[0][0], state[1][1], state[2][2]],
        [state[2][0], state[1][1], state[0][2]],
    ]
    return [player, player, player] in win_state

def game_over(state):
    return wins(state, HUMAN) or wins(state, COMP)

def empty_cells(state):
    cells = []
    for x, row in enumerate(state):
        for y, cell in enumerate(row):
            if cell == 0:
                cells.append([x, y])
    return cells

def valid_move(x, y):
    return [x, y] in empty_cells(board)

def set_move(x, y, player):
    if valid_move(x, y):
        board[x][y] = player
        return True
    else:
        return False

def minimax(state, depth, player):
    if player == COMP:
        best = [-1, -1, -infinity]
    else:
        best = [-1, -1, +infinity]

    if depth == 0 or game_over(state):
        score = evaluate(state)
        return [-1, -1, score]

    for cell in empty_cells(state):
        x, y = cell[0], cell[1]
        state[x][y] = player
        score = minimax(state, depth - 1, -player)
        state[x][y] = 0
        score[0], score[1] = x, y

        if player == COMP:
            if score[2] > best[2]:
                best = score  # max value
        else:
            if score[2] < best[2]:
                best = score  # min value

    return best


In [10]:
from os import system
import platform
def clean():
    os_name = platform.system().lower()
    if 'windows' in os_name:
        system('cls')
    else:
        system('clear')

def render(state, c_choice, h_choice):
    chars = {
        -1: h_choice,
        +1: c_choice,
        0: ' '
    }
    str_line = '---------------'

    print('\n' + str_line)
    for row in state:
        for cell in row:
            symbol = chars[cell]
            print(f'| {symbol} |', end='')
        print('\n' + str_line)

def ai_turn(c_choice, h_choice):
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    clean()
    print(f'Computer turn [{c_choice}]')
    render(board, c_choice, h_choice)

    if depth == 9:
        x = choice([0, 1, 2])
        y = choice([0, 1, 2])
    else:
        move = minimax(board, depth, COMP)
        x, y = move[0], move[1]

    set_move(x, y, COMP)
    time.sleep(1)

def human_turn(c_choice, h_choice):
    depth = len(empty_cells(board))
    if depth == 0 or game_over(board):
        return

    move = -1
    moves = {
        1: [0, 0], 2: [0, 1], 3: [0, 2],
        4: [1, 0], 5: [1, 1], 6: [1, 2],
        7: [2, 0], 8: [2, 1], 9: [2, 2],
    }

    clean()
    print(f'Human turn [{h_choice}]')
    render(board, c_choice, h_choice)

    while move < 1 or move > 9:
        try:
            move = int(input('Use numpad (1..9): '))
            coord = moves[move]
            can_move = set_move(coord[0], coord[1], HUMAN)

            if not can_move:
                print('Bad move')
                move = -1
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')


In [11]:

def main():
    clean()
    h_choice = ''
    c_choice = ''
    first = ''

    while h_choice != 'O' and h_choice != 'X':
        try:
            print('')
            h_choice = input('Choose X or O\nChosen: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    if h_choice == 'X':
        c_choice = 'O'
    else:
        c_choice = 'X'

    clean()
    while first != 'Y' and first != 'N':
        try:
            first = input('First to start?[y/n]: ').upper()
        except (EOFError, KeyboardInterrupt):
            print('Bye')
            exit()
        except (KeyError, ValueError):
            print('Bad choice')

    while len(empty_cells(board)) > 0 and not game_over(board):
        if first == 'N':
            ai_turn(c_choice, h_choice)
            first = ''

        human_turn(c_choice, h_choice)
        ai_turn(c_choice, h_choice)

    if wins(board, HUMAN):
        clean()
        print(f'Human turn [{h_choice}]')
        render(board, c_choice, h_choice)
        print('YOU WIN!')
    elif wins(board, COMP):
        clean()
        print(f'Computer turn [{c_choice}]')
        render(board, c_choice, h_choice)
        print('YOU LOSE!')
    else:
        clean()
        render(board, c_choice, h_choice)
        print('DRAW!')

    with open('Tree.json', 'w+') as f:
        json.dump(parent_child, f)

    with open('child_parent.json', 'w+') as f:
        json.dump(child_parent, f)

    exit()



In [12]:
main()


Human turn [X]

---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Computer turn [O]

---------------
| X ||   ||   |
---------------
|   ||   ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| X ||   ||   |
---------------
|   || O ||   |
---------------
|   ||   ||   |
---------------
Computer turn [O]

---------------
| X ||   ||   |
---------------
| X || O ||   |
---------------
|   ||   ||   |
---------------
Human turn [X]

---------------
| X ||   ||   |
---------------
| X || O ||   |
---------------
| O ||   ||   |
---------------
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
Bad choice
